In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
reg_train = pd.read_feather('reg_train_wkday_feather')
dtype_dict = {"id": int, "date": str, "store_nbr": int, "item_nbr": int, "onpromotion": bool}
# test = pd.read_csv("test.csv", dtype=dtype_dict, parse_dates=['date'])
# reg_test = pd.read_feather('reg_test_feather')

In [3]:
X_train = reg_train[['date','store_nbr','item_nbr','onpromotion']]
y_train = reg_train['unit_sales']
X_train.head()

,date,store_nbr,item_nbr,onpromotion
0,0,25,103665,0
1,0,25,105574,0
2,0,25,105575,0
3,0,25,108079,0
4,0,25,108701,0


In [4]:
X_train.to_feather('./x_train_feather')
y_train.to_feather('./y_train_feather')

AttributeError: 'Series' object has no attribute 'to_feather'

In [22]:
reg = linear_model.RidgeCV(alphas=[0.0001, 0.001, 0.005], normalize=True)
reg.fit(X_train, y_train)

RidgeCV(alphas=[0.0001, 0.001, 0.005], cv=None, fit_intercept=True,
    gcv_mode=None, normalize=True, scoring=None, store_cv_values=False)

In [23]:
reg.alpha_

0.0001

In [24]:
y_train_pred = reg.predict(X_train)
# The coefficients
print('Coefficients: \n', reg.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_train, y_train_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_train, y_train_pred))

Coefficients: 
 [ -1.56023223e-03   6.43778549e-02   1.00824654e-06   5.58221155e+00]
Mean squared error: 553.97
Variance score: 0.01


In [10]:
test = pd.read_csv("test.csv", dtype=dtype_dict, parse_dates=['date'])
reg_test = pd.read_feather('reg_test_feather')

In [11]:
reg_test.head()

,date,store_nbr,item_nbr,onpromotion
0,1688,1,96995,False
1,1688,1,99197,False
2,1688,1,103501,False
3,1688,1,103520,False
4,1688,1,103665,False


In [12]:
y_pred = reg.predict(reg_test)
sub = pd.DataFrame({'id': test.id.values, 'unit_sales': y_pred})
sub.to_csv('./submission.csv', index=False)